In [1]:
from lightgbm.sklearn import LGBMClassifier
from sklearn.model_selection import StratifiedGroupKFold
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import joblib
from datetime import datetime
import numpy as np

import sys
sys.path.append("..")
from src.modeling import nestedcv, random_seed

random_seed(42)

# Setup and data loading

In [2]:
### Setup
ROOT = '..'
EXTENSION = 'parquet'
DATA_FOLDER = 'data'

In [3]:
train = pd.read_parquet(f'{ROOT}/{DATA_FOLDER}/train/train_base.{EXTENSION}')
train['date_decision'] = pd.to_datetime(train['date_decision'])
train.head(2)

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0


In [4]:
# Balancing taking the number of positive target as reference
train_balanced = train.groupby('target', group_keys=False).apply(
    lambda x: x.sample(
        min(len(x), train.target.value_counts()[1])
        )
    ).reset_index(drop=True)

train_balanced.shape

C:\Users\repo7\AppData\Local\Temp\ipykernel_11116\4156781601.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_balanced = train.groupby('target', group_keys=False).apply(


(95988, 5)

In [5]:
# Import static features
train_static_0 = pd.read_parquet(f'{ROOT}/{DATA_FOLDER}/train/train_static_0_0.{EXTENSION}')
train_static_1 = pd.read_parquet(f'{ROOT}/{DATA_FOLDER}/train/train_static_0_1.{EXTENSION}')
train_static_concat = pd.concat([train_static_0, train_static_1])

# Getting rid of high missing columns
filter_high_na = (train_static_concat.isna().sum()/train_static_concat.shape[0]) < 0.5
columns_to_keep = train_static_concat.columns[filter_high_na.values]

train_merged = train_balanced.merge(train_static_concat[columns_to_keep], on='case_id', validate='1:1')

# Exclude object columns
train_merged = train_merged.loc[:, ~train_merged.columns.isin(train_merged.select_dtypes(include=['O', '<M8[ns]']).columns)]

train_merged.head()

,case_id,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,...,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,totaldebt_9A,totalsettled_863A
0,1653686,201911,47,0,0.0,0.000,1769.6000,0.0,0.0,0.0,...,24.0,0.0,0.0,0.0,29496.0,0.0,0.0,0.0,0.0,291421.2
1,1726009,201912,52,0,0.0,15385.469,1685.6000,0.0,0.0,0.0,...,24.0,0.0,0.0,0.0,28096.0,0.0,2.0,0.0,0.0,34062.0
2,736389,201907,26,0,0.0,NaN,2297.0000,0.0,0.0,0.0,...,12.0,0.0,0.0,0.0,22198.0,0.0,1.0,NaN,0.0,0.0
3,1027343,202010,91,0,NaN,NaN,2143.4001,0.0,0.0,0.0,...,12.0,0.0,0.0,0.0,17516.0,0.0,0.0,NaN,0.0,0.0
4,117009,201903,10,0,0.0,18596.201,1236.2001,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,31960.0


In [6]:
# Save columns for inference later
pd.Series([i for i in train_merged.columns if i != 'target'], name='cols').to_csv('{}/{}/output/columns_to_keep.csv'.format(ROOT, DATA_FOLDER), index=False)

In [7]:
x = train_merged.drop(['case_id', 'target'], axis=1)
y = train_merged['target']
weeks = train_merged["WEEK_NUM"]

# Nested cross validation

In [8]:
cv_inner = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)          
cv_outer = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

In [9]:
random_seed(42)

def create_params(trial):
    return {
        'verbose': -1,
        'subsample': trial.suggest_float('subsample', 0.1, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 0.95, step=0.05),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 0.95, step=0.05),
        'learning_rate': trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 30),
    }

classifier = LGBMClassifier(n_estimators=1000, objective='binary', metric='auc', n_jobs=-1) 

lgb_clfs, lgb_oof_preds, lgb_oof_targets = nestedcv(x=x,
                                                    y=y,
                                                    groups=weeks,
                                                    classifier=classifier,
                                                    cv_outer=cv_outer,
                                                    cv_inner=cv_inner,
                                                    p_grid=create_params,
                                                    n_iter=15)

[I 2024-02-20 07:43:46,152] A new study created in memory with name: no-name-e7a6b289-8c89-4f78-bd9b-bb56d7ea0a60



Nested CV: 1 of 5 outer fold


[I 2024-02-20 07:44:01,278] Trial 0 finished with value: 0.7687175753868335 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'feature_fraction': 0.8, 'learning_rate': 0.006251373574521752, 'max_depth': 6}. Best is trial 0 with value: 0.7687175753868335.
[I 2024-02-20 07:44:22,323] Trial 1 finished with value: 0.7706064455573061 and parameters: {'subsample': 0.22479561626896213, 'bagging_fraction': 0.4, 'feature_fraction': 0.9, 'learning_rate': 0.006358358856676255, 'max_depth': 22}. Best is trial 1 with value: 0.7706064455573061.
[I 2024-02-20 07:44:44,713] Trial 2 finished with value: 0.7376648080581714 and parameters: {'subsample': 0.11646759543664197, 'bagging_fraction': 0.95, 'feature_fraction': 0.8500000000000001, 'learning_rate': 0.0004335281794951569, 'max_depth': 7}. Best is trial 1 with value: 0.7706064455573061.
[I 2024-02-20 07:45:06,671] Trial 3 finished with value: 0.7566188962656676 and parameters: {'subsample': 0.24672360788274705, 'bagging_fract

Found new best score with score 0.776358


[I 2024-02-20 07:46:48,439] Trial 0 finished with value: 0.779439548942004 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'feature_fraction': 0.8, 'learning_rate': 0.006251373574521752, 'max_depth': 6}. Best is trial 0 with value: 0.779439548942004.
[I 2024-02-20 07:46:57,154] Trial 1 finished with value: 0.7820899645859425 and parameters: {'subsample': 0.22479561626896213, 'bagging_fraction': 0.4, 'feature_fraction': 0.9, 'learning_rate': 0.006358358856676255, 'max_depth': 22}. Best is trial 1 with value: 0.7820899645859425.
[I 2024-02-20 07:47:05,725] Trial 2 finished with value: 0.7474278052253305 and parameters: {'subsample': 0.11646759543664197, 'bagging_fraction': 0.95, 'feature_fraction': 0.8500000000000001, 'learning_rate': 0.0004335281794951569, 'max_depth': 7}. Best is trial 1 with value: 0.7820899645859425.
[I 2024-02-20 07:47:14,165] Trial 3 finished with value: 0.7664056290207658 and parameters: {'subsample': 0.24672360788274705, 'bagging_fractio

...

Found new best score with score 0.785076


[I 2024-02-20 08:29:12,075] Trial 0 finished with value: 0.7768534865350847 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'feature_fraction': 0.8, 'learning_rate': 0.006251373574521752, 'max_depth': 6}. Best is trial 0 with value: 0.7768534865350847.
[I 2024-02-20 08:29:22,411] Trial 1 finished with value: 0.778182607714787 and parameters: {'subsample': 0.22479561626896213, 'bagging_fraction': 0.4, 'feature_fraction': 0.9, 'learning_rate': 0.006358358856676255, 'max_depth': 22}. Best is trial 1 with value: 0.778182607714787.
[I 2024-02-20 08:29:32,679] Trial 2 finished with value: 0.74270176056653 and parameters: {'subsample': 0.11646759543664197, 'bagging_fraction': 0.95, 'feature_fraction': 0.8500000000000001, 'learning_rate': 0.0004335281794951569, 'max_depth': 7}. Best is trial 1 with value: 0.778182607714787.
[I 2024-02-20 08:29:42,597] Trial 3 finished with value: 0.7601838016256702 and parameters: {'subsample': 0.24672360788274705, 'bagging_fraction':

Found new best score with score 0.785118


[I 2024-02-20 08:33:21,566] Trial 0 finished with value: 0.7777782050986928 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'feature_fraction': 0.8, 'learning_rate': 0.006251373574521752, 'max_depth': 6}. Best is trial 0 with value: 0.7777782050986928.
[I 2024-02-20 08:33:31,615] Trial 1 finished with value: 0.7799822241931006 and parameters: {'subsample': 0.22479561626896213, 'bagging_fraction': 0.4, 'feature_fraction': 0.9, 'learning_rate': 0.006358358856676255, 'max_depth': 22}. Best is trial 1 with value: 0.7799822241931006.
[I 2024-02-20 08:33:42,396] Trial 2 finished with value: 0.7456089106592174 and parameters: {'subsample': 0.11646759543664197, 'bagging_fraction': 0.95, 'feature_fraction': 0.8500000000000001, 'learning_rate': 0.0004335281794951569, 'max_depth': 7}. Best is trial 1 with value: 0.7799822241931006.
[I 2024-02-20 08:33:53,071] Trial 3 finished with value: 0.7640345240370523 and parameters: {'subsample': 0.24672360788274705, 'bagging_fract


        Test AUC                           : 0.787
        


In [13]:
random_seed(42)

def create_params(trial):
    return {
        'subsample': trial.suggest_float('subsample', 0.1, 0.9),
        'tree_method': 'hist',
        'colsample_bytree': trial.suggest_float('bagging_fraction', 0.4, 0.95, step=0.05),
        'learning_rate': trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 30)
    }

classifier = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', eval_metric='auc', n_jobs=-1) 

xb_clfs, xb_oof_preds, xb_oof_targets = nestedcv(x=x,
                                                 y=y,
                                                 groups=weeks,
                                                 classifier=classifier,
                                                 cv_outer=cv_outer,
                                                 cv_inner=cv_inner,
                                                 p_grid=create_params,
                                                 n_iter=10)

[I 2024-02-20 10:00:15,117] A new study created in memory with name: no-name-d17b40c5-742e-4008-b48e-01eddc341a6b



Nested CV: 1 of 5 outer fold


[I 2024-02-20 10:00:48,748] Trial 0 finished with value: 0.7720008546229552 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'learning_rate': 0.015702970884055395, 'max_depth': 19}. Best is trial 0 with value: 0.7720008546229552.
[I 2024-02-20 10:01:13,152] Trial 1 finished with value: 0.7591483842044509 and parameters: {'subsample': 0.22481491235394924, 'bagging_fraction': 0.45, 'learning_rate': 0.00014936568554617635, 'max_depth': 27}. Best is trial 0 with value: 0.7720008546229552.
[I 2024-02-20 10:02:13,525] Trial 2 finished with value: 0.7617930867667851 and parameters: {'subsample': 0.5808920093945671, 'bagging_fraction': 0.8, 'learning_rate': 0.00011527987128232407, 'max_depth': 30}. Best is trial 0 with value: 0.7720008546229552.
[I 2024-02-20 10:02:18,056] Trial 3 finished with value: 0.7466946817082256 and parameters: {'subsample': 0.7659541126403374, 'bagging_fraction': 0.5, 'learning_rate': 0.0003511356313970409, 'max_depth': 7}. Best is trial 0 wit

Found new best score with score 0.772001


[I 2024-02-20 10:04:24,774] Trial 0 finished with value: 0.7826464575503282 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'learning_rate': 0.015702970884055395, 'max_depth': 19}. Best is trial 0 with value: 0.7826464575503282.
[I 2024-02-20 10:04:46,992] Trial 1 finished with value: 0.7709839979509662 and parameters: {'subsample': 0.22481491235394924, 'bagging_fraction': 0.45, 'learning_rate': 0.00014936568554617635, 'max_depth': 27}. Best is trial 0 with value: 0.7826464575503282.
[I 2024-02-20 10:05:44,212] Trial 2 finished with value: 0.7747147437810732 and parameters: {'subsample': 0.5808920093945671, 'bagging_fraction': 0.8, 'learning_rate': 0.00011527987128232407, 'max_depth': 30}. Best is trial 0 with value: 0.7826464575503282.
[I 2024-02-20 10:05:48,736] Trial 3 finished with value: 0.7573604661728557 and parameters: {'subsample': 0.7659541126403374, 'bagging_fraction': 0.5, 'learning_rate': 0.0003511356313970409, 'max_depth': 7}. Best is trial 0 wit

...

Found new best score with score 0.779460


[I 2024-02-20 11:19:28,649] Trial 0 finished with value: 0.775857680544366 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'learning_rate': 0.015702970884055395, 'max_depth': 19}. Best is trial 0 with value: 0.775857680544366.
[I 2024-02-20 11:19:50,533] Trial 1 finished with value: 0.7617738350613923 and parameters: {'subsample': 0.22481491235394924, 'bagging_fraction': 0.45, 'learning_rate': 0.00014936568554617635, 'max_depth': 27}. Best is trial 0 with value: 0.775857680544366.
[I 2024-02-20 11:20:47,551] Trial 2 finished with value: 0.766679074293054 and parameters: {'subsample': 0.5808920093945671, 'bagging_fraction': 0.8, 'learning_rate': 0.00011527987128232407, 'max_depth': 30}. Best is trial 0 with value: 0.775857680544366.
[I 2024-02-20 11:20:52,004] Trial 3 finished with value: 0.7516561477021101 and parameters: {'subsample': 0.7659541126403374, 'bagging_fraction': 0.5, 'learning_rate': 0.0003511356313970409, 'max_depth': 7}. Best is trial 0 with val

Found new best score with score 0.780471


[I 2024-02-20 11:30:27,413] Trial 0 finished with value: 0.7769158105426885 and parameters: {'subsample': 0.39963209507789, 'bagging_fraction': 0.95, 'learning_rate': 0.015702970884055395, 'max_depth': 19}. Best is trial 0 with value: 0.7769158105426885.
[I 2024-02-20 11:30:50,886] Trial 1 finished with value: 0.7629499525662949 and parameters: {'subsample': 0.22481491235394924, 'bagging_fraction': 0.45, 'learning_rate': 0.00014936568554617635, 'max_depth': 27}. Best is trial 0 with value: 0.7769158105426885.
[I 2024-02-20 11:31:51,520] Trial 2 finished with value: 0.7672015357386192 and parameters: {'subsample': 0.5808920093945671, 'bagging_fraction': 0.8, 'learning_rate': 0.00011527987128232407, 'max_depth': 30}. Best is trial 0 with value: 0.7769158105426885.
[I 2024-02-20 11:31:56,074] Trial 3 finished with value: 0.7488057766396702 and parameters: {'subsample': 0.7659541126403374, 'bagging_fraction': 0.5, 'learning_rate': 0.0003511356313970409, 'max_depth': 7}. Best is trial 0 wit


        Test AUC                           : 0.783
        


# Output and model savings

In [14]:
# Counter for current lgb version
n = 1
timestamp =datetime.today().strftime('%Y%m%d%H%M')

oof_lgb = np.concatenate(lgb_oof_preds)
true_lgb = np.concatenate(lgb_oof_targets)
auc_cv_lgb = roc_auc_score(true_lgb, oof_lgb)

print('Overall OOF AUC LGBM = {:.5f}'.format(auc_cv_lgb))

results = pd.DataFrame({'oof': oof_lgb, 'truth': true_lgb})
results.to_csv('{}/{}/output/oof_lgb_{}_{}.csv'.format(ROOT,
                                                       DATA_FOLDER,
                                                       n,
                                                       timestamp), index=False)

for model in lgb_clfs:
    joblib.dump(model, '{}/{}/models/lgb_{}_{}.pkl'.format(ROOT,
                                                           DATA_FOLDER,
                                                           n,
                                                           datetime.today().strftime('%Y%m%d%H%M')))

Overall OOF AUC LGBM = 0.78637


In [15]:
# Counter for current xgb version
n = 1
timestamp =datetime.today().strftime('%Y%m%d%H%M')

oof_xb = np.concatenate(xb_oof_preds)
true_xb = np.concatenate(xb_oof_targets)
auc_cv_xb = roc_auc_score(true_xb, oof_xb)

print('Overall OOF AUC XGB = {:.5f}'.format(auc_cv_xb))

results = pd.DataFrame({'oof': oof_xb, 'truth': true_xb})
results.to_csv('{}/{}/output/oof_xb_{}_{}.csv'.format(ROOT,
                                                      DATA_FOLDER,
                                                      n,
                                                      timestamp), index=False)

for model in xb_clfs:
    joblib.dump(model, '{}/{}/models/xb_{}_{}.pkl'.format(ROOT,
                                                          DATA_FOLDER,
                                                          n,
                                                          datetime.today().strftime('%Y%m%d%H%M')))

Overall OOF AUC XGB = 0.78023
